# Part 1 Data Process
_Readme_

The purpose of this notebook is to process the data with different timestamps.
## In a word, this notebook merges all the data into one single dense table with consistent timestamp.

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model as lm
import math
import matplotlib.pyplot as plt
%matplotlib inline
import operator

In [3]:
# Read the file
file = pd.ExcelFile('FinalDataX.xlsx')

In [4]:
i = 0
for sheetName in file.sheet_names:
    df = file.parse(sheetName)
    df.index = df.Date
    df.drop('Date',1, inplace=True)
    if i==0:
        data = df.copy()
    else:
        data = pd.concat([data,df],join = 'outer', axis = 1)
    i = i+1

# file_additional = pd.ExcelFile('additional data.xlsx')
# df_add = file_additional.parse('Additional Data')
# df_add.columns = ['Date','MSCI EM', 'MSCI EAFE','MSCI World','S&P 500','Russell 1000','Russell 2000','Russell 3000']
# df_add.index = df.Date
# df_add.drop('Date',1, inplace=True)

# data = pd.concat([data,df_add],join = 'outer', axis = 1)

# Sort the data by time
data.sort_index(inplace=True)

data['Date'] = data.index

In [5]:
monthlyDate = file.parse('Inflation')[['Date']]

In [6]:
beginDate = '1995-01-01'
endDate = '2017-01-31'
horizon = monthlyDate.loc[np.logical_and(monthlyDate.Date >= beginDate, monthlyDate.Date<=endDate)]['Date'].values

In [7]:
# data_pickPeriod = data.loc[np.logical_and(data.index>=beginDate,data.index<=endDate)]
data_pickPeriod = data.copy()

data_pickPeriod.interpolate(method='linear', axis=0, limit=None, inplace=True, limit_direction='forward', downcast=None)

In [8]:
data_pickPeriod = data_pickPeriod.loc[np.in1d(data[['Date']],(horizon)),]

In [9]:
data_pickPeriod.head(10)

,MXWO0CD,MXWO0CS,MXWO0EN,MXWO0FN,MXWO0HC,MXWO0IN,MXWO0IT,MXWO0MT,MXWO0TC,MXWO0UT,...,FTSE,FTSEMIB,CPI,Unemployment,CommAgricultural,CommEnergy,CommFood&Beverage,CommInduInput,CommMetal,Date
Date,,,,,,,,,,,,,,,,,,,,,
1995-01-31,57.250,45.51760,56.72750,58.84960,32.4307,77.380,28.09800,99.740,43.20,62.030,...,2991.60,NaN,150.5,5.6,120.88,34.04,102.23,98.77,82.79,1995-01-31
1995-02-28,57.730,46.42830,57.55330,59.50450,33.2798,77.100,29.39720,100.510,43.18,63.640,...,3009.30,NaN,150.9,5.4,123.39,35.06,103.35,96.61,77.27,1995-02-28
1995-03-31,61.130,48.80640,60.92490,61.79200,34.8582,80.610,31.61210,105.410,43.81,64.880,...,3137.90,NaN,151.2,5.4,129.73,35.07,104.77,97.99,75.07,1995-03-31
1995-04-30,61.500,49.93535,63.53690,64.67400,35.7537,83.295,33.91800,109.345,44.38,67.265,...,3218.55,NaN,151.8,5.8,132.15,37.37,103.59,99.57,76.04,1995-04-30
1995-05-31,61.550,51.21780,63.77670,65.69340,36.2112,82.110,34.54360,107.450,45.06,68.790,...,3319.40,NaN,152.1,5.6,130.63,36.89,103.84,97.37,73.35,1995-05-31
1995-06-30,61.910,51.36670,61.93600,63.68520,37.5657,81.220,37.00400,106.050,45.79,68.090,...,3314.60,NaN,152.4,5.6,125.36,35.34,106.51,96.45,75.56,1995-06-30
1995-07-31,65.110,52.21710,64.03660,67.64690,38.9456,85.810,39.75120,112.440,46.86,69.380,...,3463.30,NaN,152.6,5.7,118.36,33.24,109.08,95.27,78.59,1995-07-31
1995-08-31,63.490,51.04680,61.52920,65.22910,38.3493,83.870,39.81060,109.120,47.78,66.590,...,3477.80,NaN,152.9,5.7,113.43,33.80,108.13,93.44,78.99,1995-08-31
1995-09-30,63.825,53.50650,62.29175,67.09355,41.0170,84.145,39.79855,109.550,50.43,68.740,...,3514.20,NaN,153.1,5.6,113.71,34.30,107.65,91.19,74.93,1995-09-30


#### Now that the table is dense, let's check what we have

In [10]:
data.columns

Index(['MXWO0CD', 'MXWO0CS', 'MXWO0EN', 'MXWO0FN', 'MXWO0HC', 'MXWO0IN',
       'MXWO0IT', 'MXWO0MT', 'MXWO0TC', 'MXWO0UT', 'MXEF0CD', 'MXEF0CS',
       'MXEF0EN', 'MXEF0FN', 'MXEF0HC', 'MXEF0IN', 'MXEF0IT', 'MXEF0MT',
       'MXEF0TC', 'MXEF0UT', 'MZUSL', 'MZUSM', 'MZUSS', 'MXEALC', 'MXEAMC',
       'MXEASC', 'MXEFLC', 'MXEFMC', 'MXEFSC', 'FED', 'FED_Eff', 'Change',
       'LIBOR1Y', 'USTreasury', 'USCorporate', 'AsianPacificAgg', 'EuroAgg',
       'EMAgg', 'MXEF', 'MXEA ', 'MSCI', 'SPX', 'RUI', 'RUT', 'SXXP', 'RUA',
       'NI225', 'SPTSX', 'HSI', 'SHCOMP', 'DAX', 'CAC', 'FTSE', 'FTSEMIB',
       'CPI', 'Unemployment', 'CommAgricultural', 'CommEnergy',
       'CommFood&Beverage', 'CommInduInput', 'CommMetal', 'Date'],
      dtype='object')

Export to a file so that everyone can use

In [11]:
writer = pd.ExcelWriter('mergedData_v4.xlsx')
data_pickPeriod.to_excel(writer,'Sheet1')
writer.save()